<h2>FT005</h2>
<p> En este apartado se trata la base de datos FT005 </p>

In [1]:
#Cargamos los paquetes que vamos a necesitar
import pandas as pd
import numpy as np
import pyreadstat
import pyarrow.feather as feather

In [2]:
# Se carga el FT001 completo desde el archivo .csv
FT005 = pd.read_csv('C:/Users/Miguel Angel/OneDrive - Supersalud/Supersalud/Bases_FT/BD_actualizadas/FT005_1219_0521.csv', dtype={'Nit':object, 'Año': object, 'Periodo': object, 'Fecha': object})

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
FT005.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12884322 entries, 0 to 12884321
Data columns (total 14 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   Nit              object
 1   RazonSocial      object
 2   Año              object
 3   Periodo          object
 4   Fecha            object
 5   lineaNegocio     int64 
 6   tipoIdProveedor  object
 7   idProveedor      object
 8   dvProveedor      int64 
 9   nombreProveedor  object
 10  codigoConcepto   int64 
 11  descripcionCyG   int64 
 12  valorCausado     int64 
 13  valorPagado      int64 
dtypes: int64(6), object(8)
memory usage: 1.3+ GB


In [4]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT005.rename(columns={'Nit': 'NIT_EPS', 'Año': 'Ano', 'Periodo': 'Mes', 'lineaNegocio': 'Linea','tipoIdProveedor': 'Tipoproveedor', 'idProveedor': 'Nitproveedor', 'codigoConcepto': 'Concepto', 'descripcionCyG': 'CyG', 'valorCausado': 'VCausado', 'valorPagado': 'VPagado'}, inplace=True)

In [5]:
#Se transforman las variables a STR para mejorar su manipulación
FT005[['NIT_EPS', 'Nitproveedor', 'Tipoproveedor', 'Mes', 'Ano']] = FT005[['NIT_EPS', 'Nitproveedor', 'Tipoproveedor', 'Mes', 'Ano']].astype(str)

In [6]:
# Se filtra la línea de negocio de interés. En este caso corresponde a la línea 1
FT005 = FT005.loc[FT005['Linea'] == 1]

In [7]:
# Para hacer la suma de Valor pagado y Valor causado por NIT EPS, tipo de proveedor, NIT proveedor y Mes de interés
FT005 = FT005.groupby(['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano']).agg({'VPagado': 'sum','VCausado': 'sum'}).reset_index()

In [8]:
# Se comprueba que no haya duplicados cuando se filtra por NIT EPS, tipo de proveedor, NIT proveedor y Mes. Para esto se cuenta el número de duplicados
duplicadosFT005 = FT005[FT005.duplicated(['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano'], keep='last')]
print("El número de registros duplicados es:", duplicadosFT005.NIT_EPS.count(), sep='\n')

El número de registros duplicados es:
0


In [9]:
# Se convierte la variable en mes y año
FT005[['Ano', 'Mes']] = FT005[['Ano', 'Mes']].astype(float).astype(int)
# Se agrega la información por NIT_EPS, Nitproveedor, noID, Departamento, Ano, Mes para las variables VPagado, VCausado, PW, Giro, No_UPC, Compra_Cartera
FT005 = FT005.groupby(['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano'])[['VPagado', 'VCausado']].sum().reset_index()

In [10]:
# Se crea el rango de meses necesarios en cada mes
fecha_ideal=range(1, 12 + 1)
# Se rellena con ceros la información para los meses faltantes
FT005 = FT005.set_index('Mes').groupby(['NIT_EPS', 'Nitproveedor', 'Ano']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['NIT_EPS', 'Nitproveedor', 'Ano'], 1)

In [11]:
# Se desagrupa la información
FT005 = FT005.reset_index()

In [12]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT005[
    (FT005['Ano'] == 2019) & (FT005['Mes'] == 1) |
    (FT005['Ano'] == 2019) & (FT005['Mes'] == 2) |
    (FT005['Ano'] == 2019) & (FT005['Mes'] == 3) |
    (FT005['Ano'] == 2019) & (FT005['Mes'] == 4) |
    (FT005['Ano'] == 2019) & (FT005['Mes'] == 5) |
    (FT005['Ano'] == 2019) & (FT005['Mes'] == 6) |
    (FT005['Ano'] == 2019) & (FT005['Mes'] == 7) |
    (FT005['Ano'] == 2019) & (FT005['Mes'] == 8) |
    (FT005['Ano'] == 2019) & (FT005['Mes'] == 9) |
    (FT005['Ano'] == 2019) & (FT005['Mes'] == 10) |
    (FT005['Ano'] == 2019) & (FT005['Mes'] == 11)
    ].index

In [13]:
# Se eliminan las observaciones que cumplen las condiciones anteriores (se eliminan los meses de 2019 que no son de interés)
FT005 = FT005.drop(df_filtered)

<p> Esta línea se debe actualizar según los meses requeridos. Se elimnan los meses que no son de interés debido al volumen de los datos que se generan </p>

In [14]:
# Se encuentran las observaciones que cumplen las condiciones (se elimnan los meses de 2021 que son de interés)
df_filtered = FT005[
    (FT005['Ano'] == 2021) & (FT005['Mes'] == 6) |
    (FT005['Ano'] == 2021) & (FT005['Mes'] == 7) |
    (FT005['Ano'] == 2021) & (FT005['Mes'] == 8) |
    (FT005['Ano'] == 2021) & (FT005['Mes'] == 9) |
    (FT005['Ano'] == 2021) & (FT005['Mes'] == 10) |
    (FT005['Ano'] == 2021) & (FT005['Mes'] == 11)
    ].index

# Se eliminan las observaciones que cumplen las condiciones anteriores (se eliminan los meses de 2019 que no son de interés)
FT005 = FT005.drop(df_filtered)

In [15]:
# Se completan los años y los meses para las observaciones que no presentaron ninguna observación en un año de interés. Por ejemplo, a aquellos sujetos que reportaron cifras en 2019, pero no en 2020, se les completa el año 2020 para poder calcular la diferencia entre diciembre de 2019 y enero de 2020
# Se convierte la variable en mes y año
FT005[['Ano', 'Mes']] = FT005[['Ano', 'Mes']].astype(float)
FT005['periodo'] = pd.to_datetime(FT005.Ano*10000 + FT005.Mes*100 + 1, format='%Y%m%d')
fecha_ideal = pd.date_range('12-01-2019','05-01-2021',freq='M')

In [16]:
# Se agrupa la información según NIT EPS, NIT proveedor y periodo sumando el valor de VPagado y VCausado que corresponda a cada agrupación descrita
FT005 = FT005.groupby(['NIT_EPS', 'Nitproveedor', pd.Grouper(key='periodo', freq='M')])[['VPagado', 'VCausado']].sum().reset_index()

In [17]:
# Se prepara el dataframe para calcular la diferencia periodo a periodo (t+1 - t) respetando la agregación anterior
FT005 = FT005.set_index('periodo').groupby(['NIT_EPS', 'Nitproveedor']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['NIT_EPS', 'Nitproveedor'], 1).reset_index()

In [18]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT005.rename(columns={'level_2': 'periodo'}, inplace=True)

In [19]:
# Se establece el index (identificación) como NIT EPS, NIT proveedor y el periodo para cada observación
FT005_dif = FT005.set_index(['NIT_EPS', 'Nitproveedor', 'periodo'])

In [20]:
# Se calcula la diferencia mensual para conocer el valor mensual de cada cuenta ya que por defecto vienen agregadas (t+1 - t) y se llenan los espacios vacíos con ceros
FT005_dif = FT005_dif.diff().fillna(0).reset_index()

In [21]:
#  Se extrae el mes y el año para porterior left join con el agregado
FT005_dif['Ano'] = FT005_dif['periodo'].dt.year
FT005_dif['Mes'] = FT005_dif['periodo'].dt.month

FT005['Ano'] = FT005['periodo'].dt.year
FT005['Mes'] = FT005['periodo'].dt.month

# Se eliminan las columnas que no son de interés
FT005 = FT005.drop(['periodo'], axis=1)
FT005_dif = FT005_dif.drop(['periodo'], axis=1)

In [23]:
# Se reemplazan los valores de VPagado y VCausado para el mes 1 pues la idea es que estos valores no son la diferencia del mes 12 del año inmeditamente anterior. Es decir, el valor inicial de cada año es el de enero.
FT005_dif.loc[FT005_dif['Mes'] == 1] = FT005

In [24]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT005_dif.rename(columns={'VPagado': 'pagomes', 'VCausado': 'causmes'}, inplace=True)

In [25]:
FT005_EPS = FT005_dif.groupby(['NIT_EPS', 'Mes', 'Ano']).agg({'causmes':'sum', 'pagomes':'sum'}).reset_index()
#HASTA AQUI VAMOS BIEN

In [26]:
# Se guarda un back up del FT005 antes de aplicar las diferencias
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
FT005_dif.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT005_dif_backup_3.csv', index = False, encoding='utf-8-sig')
FT005.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT005_backup_3.csv', index = False, encoding='utf-8-sig')
# feather
feather.write_feather(FT005_dif, 'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT005_dif_backup_feather_3')
feather.write_feather(FT005, 'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT005_backup_feather_3') # Esta línea guarda la base de datos en binario para optimizar el tiempo de cargay el espacio

In [27]:
# Se sustituye el dataframe del FT005 para ahorrar espacio en la memoria
FT005 = FT005_dif

In [32]:
# Se eliminan los dataframes auxiliares para optimizar el procesamiento de la información
del [FT005_dif, fecha_ideal, FT005_EPS, duplicadosFT005, df_filtered]

In [ ]:
# Aquí se recupera el back up
# FT005 = pd.read_feather('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT005_dif_backup_feather_3')

In [31]:
# Se agrupa la información de interés
FT005 = FT005.groupby(['NIT_EPS', 'Nitproveedor', 'Ano', 'Mes']).agg({'pagomes':'sum', 'causmes':'sum'}).reset_index()

In [34]:
# Se carga la base de datos webgironoupc para pegarla con el FT005
webgironoupc = pd.read_excel('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/GD_UPC_CC_PW_2020_ENEABR2021_V2.xlsx', dtype={'NIT_EPS':object, 'Nitproveedor': object, 'Mes':object, 'Ano':object}) 

In [35]:
# Se convierte la variable mes y año como STR
webgironoupc[['Ano', 'Mes']] = webgironoupc[['Ano', 'Mes']].astype(float).astype(int).astype(str)
FT005[['Ano', 'Mes']] = FT005[['Ano', 'Mes']].astype(float).astype(int).astype(str)

In [36]:
# Se convierte la variable Nitproveedor como STR
webgironoupc[['Nitproveedor']] = webgironoupc[['Nitproveedor']].astype(str)
FT005[['Nitproveedor']] = FT005[['Nitproveedor']].astype(str)

In [37]:
# Se agrupa la información por NIT EPS, NIT proveedor, mes y año para la base de webgironoupc
webgironoupc = webgironoupc.groupby(['NIT_EPS', 'Nitproveedor', 'Ano', 'Mes']).agg({'Giro':'sum', 'No_UPC':'sum', 'Compra_Cartera':'sum', 'PW':'sum'}).reset_index()

In [38]:
# Se convierte la variable año, mes y Nitproveedor como STR
webgironoupc[['Ano', 'Mes']] = webgironoupc[['Ano', 'Mes']].astype(float).astype(int).astype(str)
webgironoupc[['Nitproveedor']] = webgironoupc[['Nitproveedor']].astype(str)

In [39]:
# Se unen las dos bases de datos para obtener la información de webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y FT005 en una sola base de datos
FT005 = pd.merge(FT005, webgironoupc, how='outer', left_on=['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano'], right_on=['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano'])

In [42]:
# Se eliminan los dataframes auxiliares para optimizar el procesamiento de la información
del [webgironoupc, FT005_EPS]

In [44]:
# Se renombran las columnas de interés
FT005.rename(columns={'NIT_EPS': 'nit_eps', 'Nitproveedor': 'nitproveedor', 'Ano': 'ano', 'Mes': 'mes', 'PW': 'pw', 'Giro': 'giro', 'No_UPC': 'no_upc', 'Compra_Cartera': 'compra_cartera'}, inplace=True)

In [47]:
# Se agrupa la información según NIT EPS, NIT proveedor, ano y mes
FT005 = FT005.groupby(['nit_eps', 'nitproveedor', 'ano', 'mes',]).agg({'pagomes':'sum', 'causmes':'sum', 'giro':'sum', 'no_upc':'sum', 'compra_cartera':'sum', 'pw':'sum'}).reset_index()

In [49]:
# Se convierte la variable NIT EPS, año y mes como STR
FT005[['nit_eps', 'mes', 'ano']] = FT005[['nit_eps', 'mes', 'ano']].astype(str)

In [50]:
# Guardar la base de datos lista para ser tratada en formato Excel y CSV
# FT005.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/CausaPrestador_2020_2021.xlsx', index = False)
FT005.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/CausaPrestador_2020_eneabr_2021_V1.csv', index = False, encoding='utf-8-sig')

In [53]:
FT005_EPS = FT005.groupby(['nit_eps', 'mes', 'ano']).agg({'pagomes':'sum', 'causmes':'sum', 'giro':'sum', 'no_upc':'sum', 'compra_cartera':'sum', 'pw':'sum'}).reset_index()
# #HASTA AQUI VAMOS BIEN

In [ ]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT005, FT005_EPS]